<a href="https://colab.research.google.com/github/ftmthb/NER/blob/main/gradio_ner_pfe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### installations

In [1]:
!pip install virtualenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 28.7 MB/s eta 0:00:00


In [2]:
!virtualenv /content/drive/MyDrive/virtual_env


created virtual environment CPython3.10.12.final.0-64 in 2312ms
  creator CPython3Posix(dest=/content/drive/MyDrive/virtual_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.2, setuptools==75.1.0, wheel==0.44.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [3]:
!source /content/drive/MyDrive/virtual_env/bin/activate; pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 49.6 MB/s eta 0:00:00


In [4]:
!source /content/drive/MyDrive/virtual_env/bin/activate; pip install ipymarkup

  Preparing metadata (setup.py) ... done
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26097 sha256=f17abf528f1a0c5f8b71ef57623ee800c8849496a2e8f3845993a56b7167e059
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built intervaltree


##### env activate

In [1]:
import sys
sys.path.append("/content/drive/MyDrive/virtual_env/lib/python3.10/site-packages")

#### import model

In [2]:
from transformers import pipeline

classifierx = pipeline('ner',r"Fatimetou/xlm-roberta-large",  device_map = 'auto')

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

#### Functions

##### id_2_label

In [3]:
label_2_id = {'O': 0,
 'B-FULL_NAME': 1,
 'I-FULL_NAME': 2,
 'B-CIN': 3,
 'I-CIN': 4,
 'B-PROPERTY_TITLE': 5,
 'I-PROPERTY_TITLE': 6,
 'B-PROPERTY_PRICE': 7,
 'I-PROPERTY_PRICE': 8,
 'B-SAVE_DATE': 9,
 'I-SAVE_DATE': 10}

label_2_id = {v: k for k, v in label_2_id.items()}

### Split text

In some cases where the given text is tokenized into over 512 tokens, it needs to be splitted into chunks  

In [4]:
def split_text(text, max_length=512):#, overlap=50):
    if len(text) <= max_length:
        return [(0, text)]

    chunks = []
    start = 0
    while start < len(text):
        end = start + max_length
        if end >= len(text):
            chunks.append((start, text[start:]))
            break
        chunks.append((start, text[start:end]))
        start = end# - overlap
    return chunks       #where chunk is a list of tuples with the index of the first 'string' of the chunk and the text composing it

In [5]:
def process_chunk(chunk_start, chunk_text):
    res = classifierx(chunk_text)
    spans = []
    for elt in res:
        label = elt['entity']
        id = int(label.split('_')[1])
        if id != 0:
            spans.append((
                chunk_start + elt['start'],
                chunk_start + elt['end'],
                label_2_id[id]
            ))
    return spans    #classifies ie extracts named entities by chunk and return spans like []

In [7]:
!pip install ipymarkup

  Preparing metadata (setup.py) ... done
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26096 sha256=c1a1d2871d20ad69ca810d21c97d03be1715274b1091b8356750fd9ec0768295
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built intervaltree


In [17]:
from ipymarkup import show_span_box_markup, format_span_box_markup
import concurrent.futures

# def merge_overlapping_entities(entities):
#     sorted_entities = sorted(entities, key=lambda x: x[0])
#     merged = []
#     for entity in sorted_entities:
#         if not merged:
#             merged.append(entity)
#         else:
#             last = merged[-1]
#             # If entities overlap or are adjacent and have the same label
#             if entity[0] <= last[1] + 1 and entity[2] == last[2]:
#                 merged[-1] = (last[0], max(last[1], entity[1]), last[2])
#             else:
#                 merged.append(entity)
#     return merged



from ipymarkup.palette import palette, Color, BLUE, RED, GREEN, ORANGE, PURPLE, BROWN
from ipymarkup.palette import material
from ipymarkup import format_span_box_markup

TEAL = Color(
    'teal',
    background=material('Teal', '50'),
    border=material('Teal', '100'),
    text=material('Teal', '300'),
    line=material('Teal', '200')
)

AMBER = Color(
    'amber',
    background=material('Amber', '50'),
    border=material('Amber', '100'),
    text=material('Amber', '300'),
    line=material('Amber', '200')
)

PINK = Color(
    'pink',
    background=material('Pink', '50'),
    border=material('Pink', '100'),
    text=material('Pink', '300'),
    line=material('Pink', '200')
)

CYAN = Color(
    'cyan',
    background=material('Cyan', '50'),
    border=material('Cyan', '100'),
    text=material('Cyan', '300'),
    line=material('Cyan', '200')
)

YELLOW = Color(
    'yellow',
    background=material('Yellow', '50'),
    border=material('Yellow', '100'),
    text=material('Yellow', '300'),
    line=material('Yellow', '200')
)

def get_span_box_markup(text, spans):
    custom_palette = palette(
        **{
            "B-FULL_NAME": BLUE, "I-FULL_NAME": CYAN,
            "B-CIN": RED, "I-CIN": PINK,
            "B-PROPERTY_TITLE": GREEN, "I-PROPERTY_TITLE": TEAL,
            "B-PROPERTY_PRICE": ORANGE, "I-PROPERTY_PRICE": YELLOW,
            "B-SAVE_DATE": PURPLE, "I-SAVE_DATE": AMBER,
            "B-OWNER_NAME": BROWN, "I-OWNER_NAME": BLUE,
            "B-PROPERTY_AREA": CYAN, "I-PROPERTY_AREA": RED,
            "B-TRANSACTION_DATE": TEAL, "I-TRANSACTION_DATE": PINK
        }
    )
    return ''.join(format_span_box_markup(text, spans, palette=custom_palette))

def predict(text_to_classify):
    chunks = split_text(text_to_classify)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_to_chunk = {executor.submit(process_chunk, start, text): (start, text) for start, text in chunks}
        all_spans = []
        for future in concurrent.futures.as_completed(future_to_chunk):
            all_spans.extend(future.result())

    # merged_spans = merge_overlapping_entities(all_spans)
    # return show_span_box_markup(text_to_classify, all_spans)
    return get_span_box_markup(text_to_classify, all_spans)


In [20]:
import gradio as gr
iface = gr.Interface(
    fn=predict,
    inputs=gr.Textbox(lines=10, label="Texte à classifier"),
    outputs=gr.HTML(label="Résultat"),
    title="Classification d'entités nommées pour longs textes",
    description="Entrez un texte de n'importe quelle longueur pour voir les entités nommées identifiées."
)

iface.launch(share=True, debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://98815f2c22a91663e6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://98815f2c22a91663e6.gradio.live
